In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("total_지하철_혼잡도_계절별_시간대별_강수상태별_분석.csv")
# df

In [3]:
# 0. 시간대 숫자형 변환
df['시간대'] = df['시간대'].str.extract(r'(\d+)').astype(int)

In [4]:
# 1. 출퇴근 시간대 라벨링 함수 정의
def commute_label(hour):
    if 6 <= hour <= 9:
        return '출근'
    elif 17 <= hour <= 20:
        return '퇴근'
    else:
        return None

# 2. 출퇴근 시간대 컬럼 생성
df['출퇴근시간'] = df['시간대'].apply(commute_label)

In [5]:
# 3. 출퇴근 시간대 데이터만 필터링
commute_df = df[df['출퇴근시간'].isin(['출근', '퇴근'])]

In [6]:
# 4. 그룹별 혼잡도 평균 계산 (출퇴근을 하나의 컬럼으로 유지)
grouped = commute_df.groupby(
    ['계절', '호선', '상하구분', '시간강수량_상태', '출퇴근시간']
)['혼잡도'].mean().reset_index()

In [7]:
# 5. NaN이 있을 경우 0으로 처리
grouped['혼잡도'] = grouped['혼잡도'].fillna(0)

In [8]:
# 6. 확인용 출력
print(grouped['출퇴근시간'].value_counts(dropna=False))
print(grouped.head())

출퇴근시간
퇴근    248
출근    238
Name: count, dtype: int64
   계절  호선 상하구분   시간강수량_상태 출퇴근시간        혼잡도
0  가을   1   상선     강수량 없음    출근  22.290690
1  가을   1   상선     강수량 없음    퇴근  22.325899
2  가을   1   상선  강한 비 or 눈    출근  32.402439
3  가을   1   상선  강한 비 or 눈    퇴근  18.835616
4  가을   1   상선  보통 비 or 눈    출근  20.078146


In [9]:
# 7. CSV로 저장 (출근/퇴근을 컬럼이 아닌 **행으로** 유지)
grouped.to_csv('출퇴근_혼잡도.csv', index=False, encoding='utf-8-sig')